In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [33]:
data_enero = pd.read_csv('./datasets/147_enero.csv', sep=';')
data_agosto = pd.read_csv('./datasets/147_agosto.csv', sep=';')
data_junio = pd.read_csv('./datasets/147_junio.csv', sep=';')
data = pd.concat([data_enero, data_agosto, data_junio], ignore_index=True)
poblaciones = pd.read_csv('./datasets/caba_pob_barrios_2010.csv')
delitos = pd.read_csv('./datasets/delitos_2021.csv', sep=';')

In [34]:
delitos_cant = pd.read_csv("datasets/delitos_2021_cantidad.csv", sep=";")
denuncias_cant = pd.read_csv("datasets/147_merge_denuncias.csv", sep=";")
delitos_denuncias_merge = pd.merge(
    delitos_cant, denuncias_cant, on="BARRIO", how="inner")


In [35]:
delitos_denuncias_merge = delitos_denuncias_merge.drop(['POBLACION_x'], axis=1).rename(columns={'POBLACION_y': 'POBLACION', 'CANTIDAD':'CANTIDAD_DELITOS'})



In [36]:
delitos_denuncias_merge

,BARRIO,CANTIDAD_DELITOS,PUNTAJE_x,POBLACION,CANTIDAD_RECLAMOS,PUNTAJE_y
0,PALERMO,7785,34.365702,226534,1860,0.821069
1,FLORES,4842,29.468687,164310,1192,0.725458
2,RECOLETA,4397,27.841096,157932,869,0.550237
3,CABALLITO,4046,22.978714,176076,1420,0.806470
4,ALMAGRO,3590,27.259129,131699,883,0.670468
5,BELGRANO,3030,23.996769,126267,1106,0.875922
6,BARRACAS,2864,32.017171,89452,391,0.437106
7,NUÑEZ,1424,27.411500,51949,512,0.985582
8,PUERTO MADERO,229,34.046982,6726,41,0.609575


In [37]:

denuncias_cant = denuncias_cant[denuncias_cant['BARRIO'].isin([
    'PALERMO', 'BARRACAS', 'RECOLETA', 'BELGRANO', 'CABALLITO', 'FLORES', 'ALMAGRO', 'NUÑEZ', 'PUERTO MADERO'])]


In [38]:
sum_denuncias = denuncias_cant.CANTIDAD_RECLAMOS.sum()
sum_delitos = delitos_cant.CANTIDAD.sum()

In [39]:
for i in range(0, 11):
    delitos_denuncias_merge['pond_{}'.format(i)] = (((delitos_denuncias_merge['CANTIDAD_DELITOS'] / sum_delitos)*i/10 + (
        delitos_denuncias_merge['CANTIDAD_RECLAMOS'] / sum_denuncias)*(1-i/10))/delitos_denuncias_merge['POBLACION'])*100000


In [15]:
delitos = delitos.value_counts('barrio').reset_index(
    name='CANTIDAD').rename(columns={'barrio': 'BARRIO'})
delitos.BARRIO = delitos.BARRIO.apply(lambda x: x.upper())
delitos = delitos.merge(poblaciones, on='BARRIO', how='left')
delitos['PUNTAJE'] = (delitos.CANTIDAD / delitos.POBLACION)*1000

In [16]:
data.categoria.value_counts()

LIMPIEZA Y RECOLECCIÓN                   11648
TRÁNSITO                                  8280
CALLES Y VEREDAS                          2687
BARRIOS EMERGENTES                        2441
ARBOLADO Y ESPACIOS VERDES                2287
TRÁMITES Y SERVICIOS                      1258
ALUMBRADO                                  830
ORDENAMIENTO DEL ESPACIO PÚBLICO           781
RECICLADO Y PROTECCIÓN AMBIENTAL           709
CONTROL EDILICIO, OBRAS Y CATASTRO         495
FISCALIZACIÓN ACTIVIDADES COMERCIALES      493
MEDIOS DE TRANSPORTE                       464
PLUVIALES                                  120
SEGURIDAD                                  116
SALUD Y SERVICIOS SOCIALES                 115
DESARROLLO URBANO                           26
TERRENO BALDÍO                              21
LOTBA                                        2
EDUCACION                                    1
Name: categoria, dtype: int64

In [17]:
data = data[data['domicilio_barrio'].isin([
    'PALERMO', 'BARRACAS', 'RECOLETA', 'BELGRANO', 'CABALLITO', 'FLORES', 'ALMAGRO', 'NUÑEZ', 'PUERTO MADERO'])]


In [18]:
data = data[data['categoria'].isin(['TRÁNSITO', 'LIMPIEZA Y RECOLECCIÓN', 'CALLES Y VEREDAS', 'ORDENAMIENTO DEL ESPACIO PÚBLICO','SEGURIDAD'])]

In [19]:
poblaciones = poblaciones[poblaciones['BARRIO'].isin([
    'PALERMO', 'BARRACAS', 'RECOLETA', 'BELGRANO', 'CABALLITO', 'FLORES', 'ALMAGRO', 'NUÑEZ', 'PUERTO MADERO'])]


In [20]:
data.domicilio_barrio.value_counts()
data_counts = data.domicilio_barrio.value_counts()
data_counts = data_counts.sort_index()
data_counts = data_counts.values


In [21]:
poblaciones['CANTIDAD_RECLAMOS'] = data_counts
poblaciones['PUNTAJE'] = poblaciones['CANTIDAD_RECLAMOS'] / poblaciones['POBLACION'] * 100
poblaciones = poblaciones.reset_index(drop = True)
poblaciones.to_csv('147_merge_denuncias_total.csv', sep=';', index=False)
poblaciones.sort_values(by=['PUNTAJE'], ascending=True)


,BARRIO,POBLACION,CANTIDAD_RECLAMOS,PUNTAJE
1,BARRACAS,89452,391,0.437106
8,RECOLETA,157932,869,0.550237
7,PUERTO MADERO,6726,41,0.609575
0,ALMAGRO,131699,883,0.670468
4,FLORES,164310,1192,0.725458
3,CABALLITO,176076,1420,0.806470
6,PALERMO,226534,1860,0.821069
2,BELGRANO,126267,1106,0.875922
5,NUÑEZ,51949,512,0.985582
